In [1]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import boxscoreplayertrackv2
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import playergamelogs
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.static import teams
from json import JSONDecodeError
from ast import literal_eval
import pandas as pd
import numpy as np
import random
import requests
import math
import time

In [2]:
def str_to_mins(inp):
    lst = inp.split(":")
    sec = int(lst[0])*60 + int(lst[1])
    return sec

In [3]:
teams = pd.read_csv("../common-datasets/all_teams_per_year.csv")
year_to_ids_dict = dict()
for yr in range(1997,2019):
    year_to_ids_dict[yr] = set()
    for x in teams[f"{yr}"]:
        try:
            year_to_ids_dict[yr].add(literal_eval(x)[1])
        except:
            print(x)

nan
nan
nan
nan
nan
nan
nan


In [ ]:
year_to_ids_dict

In [ ]:
gl = teamgamelog.TeamGameLog(1610612741)
gl.get_data_frames()[0]

In [ ]:
df = pd.DataFrame()

In [ ]:
for season, set_of_teams in year_to_ids_dict.items():
    print(season)
    for team_id in set_of_teams:
        time.sleep(0.36)
        successful = False
        decode_error = False
        while not successful:
            try:
                gl = teamgamelog.TeamGameLog(team_id, timeout = 5)
                successful = True
            except:
                print(f"Timeout error on {season, team_id}, taking a 3 second break before resuming")
                time.sleep(3)
        
        df = pd.concat([df, gl.get_data_frames()[0]])

In [5]:
df = pd.read_csv("../common-datasets/all_gamelogs.csv")

In [ ]:
for game_id in df["GAME_ID"].unique():
    relevant_games = df[(df['GAME_ID'] == game_id)]

In [6]:
df

,Team_ID,GAME_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,SEASON,TEAM_ID
0,1610612747,29701187,"APR 19, 1998",LAL vs. UTA,W,NaN,NaN,NaN,240,38,...,31,43,21,6,9,12,26,102,1997,1610612747
1,1610612747,29701173,"APR 17, 1998",LAL vs. DAL,W,NaN,NaN,NaN,240,47,...,35,48,34,6,9,19,14,124,1997,1610612747
2,1610612747,29701145,"APR 14, 1998",LAL @ DAL,W,NaN,NaN,NaN,240,45,...,37,56,24,5,3,16,21,111,1997,1610612747
3,1610612747,29701137,"APR 13, 1998",LAL @ SAS,W,NaN,NaN,NaN,240,37,...,43,48,21,7,5,11,22,99,1997,1610612747
4,1610612747,29701127,"APR 11, 1998",LAL @ GSW,W,NaN,NaN,NaN,240,35,...,27,47,23,6,5,21,21,96,1997,1610612747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52133,1610612765,21800074,"OCT 27, 2018",DET vs. BOS,L,4.0,1.0,0.8,240,33,...,41,47,16,4,3,11,24,89,2018,1610612765
52134,1610612765,21800063,"OCT 25, 2018",DET vs. CLE,W,4.0,0.0,1.0,240,41,...,35,48,21,4,5,20,21,110,2018,1610612765
52135,1610612765,21800049,"OCT 23, 2018",DET vs. PHI,W,3.0,0.0,1.0,265,51,...,34,46,22,4,4,8,24,133,2018,1610612765
52136,1610612765,21800031,"OCT 20, 2018",DET @ CHI,W,2.0,0.0,1.0,240,41,...,31,49,21,4,4,9,23,118,2018,1610612765


In [7]:
import random

correct = 0
wrong = 0
for game_id in df["GAME_ID"].unique():
    relevant_boxscores = df[(df['GAME_ID'] == game_id)].reset_index()
    teamA_wp = relevant_boxscores.loc[0, "W"] / (relevant_boxscores.loc[0, "W"] + relevant_boxscores.loc[0, "L"])
    teamB_wp = relevant_boxscores.loc[1, "W"] / (relevant_boxscores.loc[1, "W"] + relevant_boxscores.loc[1, "L"])
    teamA_won = relevant_boxscores.loc[0, "WL"] == "W"
    if (teamA_wp > teamB_wp and teamA_won) or (teamA_wp < teamB_wp and not teamA_won):
        correct += 1
    elif (teamA_wp > teamB_wp and not teamA_won) or (teamA_wp < teamB_wp and teamA_won):
        wrong += 1
    elif teamA_wp == teamB_wp:
        if random.uniform(0, 1) > 0.5:
            correct += 1
        else:
            wrong += 1
        
print(correct/(correct+wrong))

c:\users\2000d\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
c:\users\2000d\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


0.7093393127183605


In [ ]:
df.columns

In [ ]:
df